In [ ]:
import json
import time
import requests
from datetime import datetime
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, DateTime, Float

# Define

In [ ]:
stationUrls = [
    "A97507",
    "A21373",
    "A156526",
    "A74863",
    "A246697"
]

api_key = "d45fda67589e376593853b0fba950fc6d0dcbb65"

# Functions to get sensor data

In [ ]:
def fetch_data_from_apis(api_key):
    sensor_data = []

    for station_id in stationUrls:  # Change variable from station_url to station_id
        api_url = f"https://api.waqi.info/feed/{station_id}/?token={api_key}"

        try:
            response = requests.get(api_url)

            if response.status_code == 200:
                api_data = response.json()
                print(f"API Response Data of station {station_id}: {json.dumps(api_data, indent=4)}")

                if api_data['status'] == 'ok':
                    station_info = api_data['data']
                    sensor_info = {
                        'api_timestamp': station_info['time']['v'],  # API timestamp
                        'etl_timestamp': int(datetime.now().timestamp()),  # Current timestamp
                        'location_address': station_info['city']['name'],  # City name
                        'location_coordinates': station_info['city']['geo'],  # Latitude and longitude
                        'name': station_info['city']['name'],  # Station name
                        'pm10': station_info['iaqi'].get('pm10', {}).get('v', None),  # PM10 value
                        'pm25': station_info['iaqi'].get('pm25', {}).get('v', None),  # PM2.5 value
                        'station_id': station_id,  # Station ID
                        'ui_url': f"https://aqicn.org/station/@{station_id}/",  # User interface URL
                        'url': api_url  # API request URL
                    }
                    sensor_data.append(sensor_info)
                else:
                    print(f"API responses do not contain the expected data structures, station: {station_id}")
            else:
                print(f"API request failed with status code: {response.status_code}，station: {station_id}")
        except Exception as e:
            print(f"Error when requesting API of station {station_id}: {e}")

    return sensor_data

# Functions to insert data into the database

In [ ]:
def save_sensor_data_to_db(sensor_data):
    # Connect to the database using SQLAlchemy
    engine = create_engine('sqlite:///your_database.db')  # Replace with your actual database path
    metadata = MetaData()

    # Table schema
    pollution_sensor_data_table = Table(
        'pollution_sensor_data', metadata,
        Column('etl_timestamp', DateTime, primary_key=True, nullable=False),
        Column('station_id', String, primary_key=True, nullable=False),
        Column('api_timestamp', DateTime, nullable=False),
        Column('location_address', String, nullable=False),
        Column('latitude', Float, nullable=False),
        Column('longitude', Float, nullable=False),
        Column('name', String, nullable=False),
        Column('pm10', Integer, nullable=False),
        Column('pm25', Integer, nullable=False),
        Column('ui_url', String, nullable=False),
        Column('url', String, nullable=False)
    )

    metadata.create_all(engine)

    # Insert data into the table
    with engine.connect() as conn:
        for data in sensor_data:
            insert_stmt = pollution_sensor_data_table.insert().values(
                etl_timestamp=datetime.fromtimestamp(data['etl_timestamp']),
                station_id=data['station_id'],
                api_timestamp=datetime.fromtimestamp(data['api_timestamp']),
                location_address=data['location_address'],
                latitude=data['location_coordinates'][0],
                longitude=data['location_coordinates'][1],
                name=data['name'],
                pm10=data['pm10'],
                pm25=data['pm25'],
                ui_url=data['ui_url'],
                url=data['url']
            )
            conn.execute(insert_stmt)

# Execution

In [ ]:
if __name__ == "__main__":
    while True:
        sensor_data = fetch_data_from_apis(api_key)
        print(sensor_data)  # Print the captured data

        if sensor_data:  # Only save if there is data
            save_sensor_data_to_db(sensor_data)
            print("Sensor data saved to database.")
        else:
            print("No sensor data to save.")

        time.sleep(300)  # Pause 5 minutes (300 seconds)

API Response Data of station A97507: {
    "status": "ok",
    "data": {
        "aqi": 72,
        "idx": -97507,
        "attributions": [
            {
                "url": "https://sensor.community/",
                "name": "Citizen Science project sensor.community",
                "station": "40256"
            },
            {
                "url": "https://waqi.info/",
                "name": "World Air Quality Index Project"
            }
        ],
        "city": {
            "geo": [
                45.462,
                9.21
            ],
            "name": "Corso 22 Marzo",
            "url": "https://aqicn.org/station/@97507",
            "location": "Corso 22 Marzo, Ortica, Milan, Milano, Italy, 20135"
        },
        "dominentpol": "pm25",
        "iaqi": {
            "pm10": {
                "v": 33
            },
            "pm25": {
                "v": 72
            }
        },
        "time": {
            "s": "2024-10-15 19:58:46",
            "